In [1]:
import csv
import datetime
import json
import os

from litellm import completion
# import ollama 

In [29]:
from litellm import get_supported_openai_params
from litellm import supports_response_schema

params = get_supported_openai_params(
    model="mistral/mistral-small-latest",
)

print(f"The model support this paramaters:\n{params}")

response_schema = supports_response_schema(
    model="mistral-small-latest",
    custom_llm_provider="mistral",
)

response_schema

The model support this paramaters:
['stream', 'temperature', 'top_p', 'max_tokens', 'tools', 'tool_choice', 'seed', 'stop', 'response_format']


False

# The CSV

In [2]:
CSV_FILEPATH = "../data/csv/lista_sentencias/2021.csv"

In [3]:
#############################
# Load data from a CSV file #
#############################
def load_csv_data(filepath):
    """Load data from a CSV file."""
    with open(filepath, mode="r", newline="", encoding="utf-8") as csv_file:
        reader = csv.DictReader(csv_file)
        return list(reader)

############################
# Read content from a file #
############################
def read_file_content(file_path):
    """Read content from a file."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

############################
# Read content from a file #
############################
def read_file_content(file_path):
    """Read content from a file."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

In [4]:
rows = load_csv_data(CSV_FILEPATH)

In [5]:
rows[12]

{'#': '20',
 'Providencia': 'T-449-21',
 'link': 'https://www.corteconstitucional.gov.co/relatoria/2021/T-449-21.htm',
 'F.Sentencia': '2021-12-13',
 'F.Publicación': '2022-02-07',
 'Tema / Resumen': 'TEMA: DERECHOS A UN DEBIDO PROCESO, TRABAJO, MÍNIMO VITAL, SALUD Y SEGURIDAD SOCIAL TRABAJADORA MIGRANTE IRREGULAR EN ESTADO DE EMBARAZORESUMEN: La accionante es una ciudadana venezolana que aduce que sus derechos fundamentales fueron vulnerados por las accionadas, con ocasión de la suspensión unilateral del contrato de trabajo verbal que tenían y el no pago de los salarios y prestaciones sociales adeudados, a pesar de encontrarse en estado de gestación.  Lo anterior, al imponerle un permiso de licencia no remunerada con el fin de soslayar el pago de salarios y las demás obligaciones prestacionales durante el confinamiento obligatorio originado por la pandemia del Covid 19.  Se analizan los siguientes temas. 1º. El alcance de los derechos laborales de los migrantes irregulares desde la óp

# Select one sentencia

In [7]:
INDEX = 12
sentencia = rows[INDEX]['Providencia']
file_path = rows[INDEX]['file_path']
content = read_file_content(f"{file_path}")  

In [8]:
content

'  \n\n* * *\n\n**TEMAS-SUBTEMAS**\n\n****\n\n**Sentencia T-449/21**\n\n**ACCION DE TUTELA TRANSITORIA DE MUJER EMBARAZADA** -Reintegro y pago de\nsalarios\n\n**CONTRATO REALIDAD** -Caso en que se configuraron los presupuestos jurídicos\n\n**CONTRATO REALIDAD** -Definición\n\n**CONTRATO REALIDAD** -Determinación de sus elementos no corresponde al juez\nde tutela pero excepcionalmente puede hacerlo\n\n**CONTRATO REALIDAD** -Elementos esenciales que deben demostrarse\n\n**DERECHO A LA ESTABILIDAD LABORAL REFORZADA DE MUJER EMBARAZADA Y DURANTE EL\nPERIODO DE LACTANCIA** -Vulneración por suspensión unilateral e indefinida del\ncontrato de trabajo\n\n_(...) (la empresa) menoscabó los derechos laborales y prestacionales de su\ntrabajadora al suspender unilateral e indefinidamente el contrato de trabajo y\npor consiguiente el pago del salario y los aportes a seguridad social de su\ntrabajadora (...), dicha suspensión no cumplió con los elementos de\ncausalidad, esto es, ser libre y voluntari

In [9]:
file_path

'/Volumes/samsung/dev/sentencias/data/markdown/T/2021-T-449-21.md'

# LLM calling

In [59]:
prompt = create_prompt_2(sentencia, content)
message = [{"role": "user", "content": prompt}]

params = {
    "messages": message,
    "model": "mistral/mistral-small-latest",
    "temperature": 0.5,
}

In [60]:
response = completion(**params)
response_content = response["choices"][0]["message"]["content"]
print(response_content)

### **Análisis de la Sentencia T-793/07**

#### **Paso 1: Reflexión inicial**
La sentencia T-793/07 de la Corte Constitucional de Colombia aborda la acción de tutela presentada por José Ramón Parra Vanegas contra el Tribunal Administrativo de Antioquia y la Fiscalía General de la Nación, argumentando la vulneración de su derecho al debido proceso debido a su desvinculación sin motivación. El demandante laboró como escolta judicial en la Fiscalía General de la Nación y fue declarado insubsistente sin una justificación clara. La sentencia revisa la procedencia de la tutela contra providencias judiciales y la jurisprudencia sobre la necesidad de motivación en actos administrativos.

#### **Paso 2: Análisis detallado por propiedades**

1. **Hechos presentados:**
   - José Ramón Parra Vanegas laboró como escolta judicial desde 1993 hasta 1996, cuando fue declarado insubsistente.
   - La resolución de insubsistencia no fue motivada y el demandante no recibió copia de la misma hasta años desp

# The prompt

In [ ]:
###############################
# Create a prompt for the LLM #
###############################
def create_prompt(sentencia, content):
    """Create a prompt for the LLM."""
    schema = """
        {
        "hechos": [],
        "derechos_fundamentales_solicitados": [],
        "problema_juridico_enunciado": [],
        "problema_juridico_resuelto": [],
        "normas_juridicas_relevantes": [],
        "ratio_decidendi": [],
        "decision": {
            "resuelve": [
            {
                "numero": 1,
                "descripcion": ""
            }
            ]
        },
        "derechos_fundamentales_amparados": []
        }
    """
    prompt = f"""
        A partir de la sentencia que se proporciona, **extraiga información exhaustiva, detallada, objetiva y fiel al texto original, sin agregar opiniones ni interpretaciones**. Organice la información en formato JSON, respetando las siguientes propiedades y descripciones: \n

        1.	Hechos presentados: Liste los hechos que originan el proceso legal. Detalle cada hecho de forma cronológica, clara y concisa.\n
        2.	Derechos fundamentales solicitados: Liste los derechos fundamentales que el accionante alega como vulnerados y que solicita proteger tal como se indica en la sentencia.\n
        3.	Problema jurídico enunciado por la Corte (PJC): Identifique el problema jurídico que la Corte textualmente dice tratar en la sentencia. Si el problema no está explícito como pregunta:\n
        •	Extraiga el texto relevante donde la Corte “se cuestiona” el asunto a resolver.\n
        •	Si hay varios problemas jurídicos, indique primero el principal y luego los secundarios.\n
        4.	Problema jurídico resuelto por la Corte (PJR):Construya el problema jurídico que efectivamente resuelve la Corte. Este debe:\n
        •	Reflejar la confrontación entre principios jurídicos o derechos fundamentales.\n
        •	Ser específico y conectar los hechos con los derechos.\n
        Ejemplo:
        •	Incorrecto: ¿Se violaron los derechos de María Pérez?
        •	Correcto: ¿Debe protegerse el derecho a la salud de una niña afectada psicológicamente por un condiloma facial, aunque no haya impacto funcional directo? \n
        2.  Hechos relevantes: hacer un listado de los hechos que jurídicamente tienen relevancia en la decisión. Efectivamente, no todos los hechos tienen una directa relevancia para la resolución del caso, por lo cual no se trata aquí de contar lo que al derecho le importa de los hechos. \n
        5.	Normas jurídicas relevantes: Liste las normas jurídicas que son importantes para resolver el caso, incluyendo normas constitucionales, legales, convencionales o internacionales. \n
        6.	Ratio Decidendi (RD): Extraiga la razón de la decisión de la Corte, justificando cómo responde al problema jurídico. Construya este argumento conectando las premisas fundamentales y desarrollando la lógica de la decisión. No copie y pegue extractos; sintetice las ideas esenciales. Si hay varios problemas jurídicos, elabore una ratio decidendi para cada uno.
        7.	Decisión: Liste los puntos del resuelve de la sentencia. Organice cada punto en un arreglo numerado con la estructura:\n
        •	Número: Número del punto.\n
        •	Descripción: Texto completo del punto, tal como aparece en la sentencia.\n
        8.	Derechos fundamentales amparados: Liste los derechos fundamentales que la Corte efectivamente ampara, diferenciándolos de los solicitados.\n

        Presenta la informacion extraida en formato JSON, siguiendo este schema:\n
        {schema}\n\n
        
        **Sentencia:** {sentencia}\n
        {content}
    """

    return prompt


In [54]:
###############################
# Create a prompt for the LLM #
###############################
def create_prompt_2(sentencia, content):
    """Create a prompt for the LLM."""
    schema = """
    {
        "hechos_presentados": [],
        "derechos_fundamentales_solicitados": [],
        "problema_presenta_accionante": "",
        "problemas_juridicos_enunciados": {
            "principal": "",
            "secundarios": [] comentario: solo proporcionar si hay más de un problema enunciado
        },
        "problemas_juridicos_resueltos": {
            "principal": "",
            "secundarios": []  comentario: solo proporcionar si hay más de un problema resuelto
        },
        "hechos_relevantes": [],
        "sentencias_relevantes": [],
        "ratio_decidendi": {
            "principal": "",
            "secundarios": [] comentario: solo proporcionar si se resuelve mas de un problema
        },
        "decision": {
            "resuelve": [
                {
                    "numero": 1,
                    "descripcion": ""
                }
            ]
        },
        "derechos_fundamentales_amparados": []
    }
    """
    prompt = f"""
    A partir de la sentencia que se proporciona, **extraiga información exhaustiva, detallada, objetiva y fiel al texto original**.**Realice un análisis paso a paso**, siguiendo estas instrucciones:

    ### **Paso 1: Reflexión inicial**
    Tómese un momento para leer y comprender el contenido completo de la sentencia. Identifique las secciones relevantes que aportan información para completar cada una de las propiedades solicitadas a continuación. Piense cuidadosamente sobre cómo cada sección responde al propósito de la tarea. Esto lo ayudará a organizar sus respuestas de manera clara y lógica.

    ---

    ### **Paso 2: Análisis detallado por propiedades**
    Para cada propiedad, siga las instrucciones específicas:

    1. **Hechos presentados:**  
    - Liste los hechos que originan el proceso legal.  
    - Asegúrese de presentar los hechos de forma clara.

    2. **Derechos fundamentales solicitados:**  
    - Liste los derechos fundamentales que el accionante alega como vulnerados, tal como se indican en la sentencia.
    
    3. **Problema que presenta el accionante:**
    - A partir de los hechos presentados y los derechos fundamentales solicitados, identifique el problema jurídico que el accionante pretende que se resuelva.

    4. **Problema jurídico enunciado por la Corte (PJC):**  
    - Identifique el problema jurídico que la Corte textualmente menciona que va a tratar.  
    - Si no está explícitamente formulado como pregunta, extraiga el texto relevante donde la Corte expone el asunto a resolver.  
    - Si hay varios problemas jurídicos, seleccione el principal primero y luego los secundarios.

    5. **Problema jurídico resuelto por la Corte (PJR):**  
    - Reflexione sobre el problema jurídico realmente resuelto.
    - El problema jurídico es una pregunta que revela la confrontación específica de principios jurídicos para la resolución de un caso. Determina el quid del asunto. Un importante número de veces, el problema jurídico enunciado por la Corte (PJC) dista de aquel que ella resuelve (PJR). Es por esto que el problema jurídico que ella resuelve hay que construirlo. 
    - Partir del problema juridico enunciado por la corte e irlo depurando a medida que la misma ratio decidendi vaya mostrando qué es lo que se pregunta la Corte.   
    - Construya este problema jurídico de manera específica, conectando los hechos con los derechos fundamentales en disputa.
    - Tómese un momento para verificar que el PJR sea lógico y refleje las tensiones jurídicas reales del caso.
    - Ejemplo:
        •	Incorrecto: ¿Se violaron los derechos de María Pérez?
        •	Correcto: ¿Debe protegerse el derecho a la salud de una niña afectada psicológicamente por un condiloma facial, aunque no haya impacto funcional directo?

    6. **Hechos relevantes:**  
    - Liste únicamente los hechos que tienen verdera relevancia jurídica directa para la decisión.
    - Distinguir de los hechos presentados.  
    - No incluya información irrelevante o redundante.

    7. **Sentencias relevantes:**  
    - Liste todas sentencias de la misma corte consitucional mencionadas o referenciadas en esta sentencia, tanto setencias tipo T, C, SU y/o A.

    8. **Ratio Decidendi (RD):**  
    - Resuma la razón de la decisión de la Corte.  
    - Conecte las premisas fundamentales y desarrolle la lógica de la decisión sin copiar extractos textuales.  
    - Si hay varios problemas jurídicos, elabore una ratio decidendi para cada uno.

    9. **Decisión:**  
    - Liste los puntos del resuelve de la sentencia.  
    - Organice cada punto en un arreglo numerado con las siguientes propiedades:  
        - **Número:** Número del punto.  
        - **Descripción:** Texto completo del punto, tal como aparece en la sentencia.

    10. **Derechos fundamentales amparados:**
    - En caso que la sentencia falle a favor del accionante, liste los derechos fundamentales que la Corte efectivamente ampara en su decisión.
    - Diferencie claramente estos derechos de los derechos solicitados.
    - En el caso de que la sentencia falle en contra del accionante, deje este campo vacío.

    ---

    ### **Paso 3: Presentación estructurada**
    Presente toda la información extraída en formato JSON, siguiendo el esquema indicado:

    {schema}

    ---

    **Tómese un momento para revisar que toda la información esté completa y organizada de manera lógica.**  

    ### **Sentencia:** 
    {sentencia}

    ### **Contenido de la sentencia:**  
    {content}
    """

    return prompt
